In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())
drive = lm.get_drive(token=rt())

# Tutorial 7 - Drive

Lumipy supports the use of Luminesce's Drive providers. With these you can build queries that pull data from files in Drive (csv, xlsx and Sqlite) and also write queries that push their results to drive. 

In addition to the above Lumipy also has helper functionality for creating/deleting folders and for uploading/downloading certain sorts of files (these are shown in later sklearn examples). 

## Putting Query Results in Drive

In this example we will demonstrate how to build a query that pushes its result to Drive as a csv. This simply done by chaining on the `.to_drive(<file path in drive>)` method and then calling `.go()` to start as usual. The file type is inferred from the file type in the file path.

In [2]:
portfolio = atlas.lusid_portfolio()
qry = portfolio.select(
    '*'
).where(
    portfolio.portfolio_scope == 'Finbourne-Examples'
)
qry.to_drive(f'/test_pf_list_in_scope.csv').go()

VariableName                 FileName  RowCount
0  @tv_082543067608897702  //test_pf_list_in_scope         5

## Searching Files in Drive

There is also a means to search for files in drive using a data provider: `Drive.File` which can be found in the atlas as `atlas.drive_file`.


In [3]:
file = atlas.drive_file(recurse_depth=1)
file.select(file.full_path).where(
    file.full_path.like('%_scope.csv')
).go()

FullPath
0  /test_pf_list_in_scope.csv

## Using a Drive CSV File in a Query

Having put the results of the above query into Drive we can now query it back out. The `.use_csv_file()` method takes a file path and returns a table object that you can use the fluent syntax with as normal. 

We can see in the example below that the columns we get back match

In [4]:
csv = drive.use_csv_file('/test_pf_list_in_scope.csv')

In [5]:
atlas.lusid_portfolio

└•Provider Definition: lusid_portfolio
   ├•Table SQL Name: Lusid.Portfolio
   ├•Type: DataProvider
   ├•Category: Lusid
   ├•Description: Returns Lusid Portfolio data
   ├•Documentation: [Not Available]
   ├•Last Ping At: 2021-06-17 12:57:33.622
   └•Fields:
      ├•accounting_method              Column      Text      
      ├•as_at                          Parameter   DateTime  
      ├•base_currency                  Column      Text      
      ├•corporate_action_source_id     Column      Text      
      ├•corporate_action_source_scope  Column      Text      
      ├•description                    Column      Text      
      ├•display_name                   Column      Text      
      ├•effective_at                   Parameter   DateTime  
      ├•origin_portfolio_code          Column      Text      
      ├•origin_portfolio_scope         Column      Text      
      ├•portfolio_code                 Column      Text      
      ├•portfolio_scope                Column      Text   

In [6]:
csv.get_columns()

[└•Text ⬅ SQL Piece: [AccountingMethod]
    └•[COLUMN INPUT]
       @get_531338256127623910: AccountingMethod,
 └•Text ⬅ SQL Piece: [BaseCurrency]
    └•[COLUMN INPUT]
       @get_531338256127623910: BaseCurrency,
 └•Double ⬅ SQL Piece: [CorporateActionSourceId]
    └•[COLUMN INPUT]
       @get_531338256127623910: CorporateActionSourceId,
 └•Double ⬅ SQL Piece: [CorporateActionSourceScope]
    └•[COLUMN INPUT]
       @get_531338256127623910: CorporateActionSourceScope,
 └•Double ⬅ SQL Piece: [Description]
    └•[COLUMN INPUT]
       @get_531338256127623910: Description,
 └•Text ⬅ SQL Piece: [DisplayName]
    └•[COLUMN INPUT]
       @get_531338256127623910: DisplayName,
 └•Text ⬅ SQL Piece: [OriginPortfolioCode]
    └•[COLUMN INPUT]
       @get_531338256127623910: OriginPortfolioCode,
 └•Text ⬅ SQL Piece: [OriginPortfolioScope]
    └•[COLUMN INPUT]
       @get_531338256127623910: OriginPortfolioScope,
 └•Text ⬅ SQL Piece: [PortfolioCode]
    └•[COLUMN INPUT]
       @get_5313382561276239

In [7]:
csv.select('*').where(
    csv.portfolio_code.like('%bond%')
).go()

AccountingMethod BaseCurrency  CorporateActionSourceId  \
0          Default          USD                      NaN   
1          Default          USD                      NaN   

   CorporateActionSourceScope  Description             DisplayName  \
0                         NaN          NaN  US Corporate Bond Fund   
1                         NaN          NaN  US$ Treasury Bond Fund   

                           OriginPortfolioCode OriginPortfolioScope  \
0  fbn-qa/Finbourne-Examples/US-Corporate-Bond   Finbourne-Examples   
1   fbn-qa/Finbourne-Examples/US-Treasury-Bond   Finbourne-Examples   

       PortfolioCode      PortfolioScope PortfolioType  SubHoldingKeys  
0  US-Corporate-Bond  Finbourne-Examples   Transaction             NaN  
1   US-Treasury-Bond  Finbourne-Examples   Transaction             NaN

## Other Formats
Other file types in drive can also be used as data providers: `xlsx` and `sqlite`. These are used with the `drive.use_excel_file()` and `drive.use_sqlite_file()` methods respectively.